In [1]:
import pandas as pd
import numpy as np
import math


In [3]:
generalized_data_gt_50k = pd.read_csv('hw1-1-generalized_data_gt_50k_3_3_4_5.csv')
generalized_data_le_50k = pd.read_csv('hw1-1-generalized_data_le_50k_2_3_4_5.csv')


In [6]:
def calculate_entropy(group):
    """ Calculate entropy for a single group of records """
    if len(group) == 0:
        return 0
    value_counts = group.value_counts()
    probabilities = value_counts / value_counts.sum()
    entropy = -np.sum(probabilities * np.log(probabilities))
    return entropy

def check_l_diversity(data, l):
    """ Check if the dataset satisfies l-diversity """
    entropy_threshold = math.log(l)  # Define the minimum entropy threshold
    fails = 0  # Track number of failures
    
    # Group data by QIs and calculate entropy for each group
    grouped = data.groupby(['age', 'education', 'marital_status', 'race'])
    for name, group in grouped:
        entropy = calculate_entropy(group['occupation'])
        if entropy < entropy_threshold:
            print(f'Group {name} fails to meet the entropy l-diversity with entropy {entropy:.4f}')
            fails += 1
    if fails == 0:
        print("All groups meet the l-diversity requirement.")
        return True
    else:
        print(f"{fails} groups fail to meet the l-diversity requirement.")
        return False

l = 3
print("Checking >50K dataset for l-diversity:")
satisfied_l_diversity_gt_50k = check_l_diversity(generalized_data_gt_50k, l)


Checking >50K dataset for l-diversity:
Group ('All Ages', 'Advanced Degree', 'Married', 'Non-Western Origin') fails to meet the entropy l-diversity with entropy 1.0453
Group ('All Ages', 'Advanced Degree', 'Not-Married', 'Non-Western Origin') fails to meet the entropy l-diversity with entropy 0.6829
2 groups fail to meet the l-diversity requirement.


In [7]:

print("Checking <=50K dataset for l-diversity:")
satisfied_l_diversity_le_50k = check_l_diversity(generalized_data_le_50k, l)

Checking <=50K dataset for l-diversity:
All groups meet the l-diversity requirement.
